In [1]:
import gpflow
import tensorflow as tf
import numpy as np
import matplotlib #
from gpflow.utilities import print_summary
import pandas as pd
gpflow.config.set_default_summary_fmt("notebook")
import warnings
import glob
import random
import time
import re
import os, sys
import csv


# The lines below are specific to the notebook format
%matplotlib inline
params = {'legend.fontsize': 'x-large',
          'figure.figsize': (12, 10),
         'axes.labelsize': 'x-large',
         'axes.titlesize':'x-large',
         'xtick.labelsize':'x-large',
         'ytick.labelsize':'x-large'}
matplotlib.rcParams.update(params)
plt = matplotlib.pyplot

In [ ]:
# The global_kernel class is to find GP kernels which are suitable for global stellar parameters (e.g. Teff, Radius) 
#and resample them as a function of the age. 
class global_2d_kernel:
    '''
    The global_kernel class aims to find an proper and efficient GP kernel for a global parameter (e. g. Teff) for
    the whole grid.  
    '''
    
    def __init__(self, datapath = None, savepath = None):
        return None
    
    def set_path(self, datapath = None, savepath = None):
        self._datapath = datapath
        self._savepath = savepath
        
        if not os.path.exists(datapath):
            raise Warning(f'datapath:' + datapath + ' does not exist')
        
        if not os.path.exists(savepath): os.makedirs(savepath)
        
        print('Data path is set as' + self._datapath)
        print('Save path is set as' + self._savepath)
        return self
    
    #############################################################################################################
    ################Change this function for different data formats##############################################
    #############################################################################################################
    def get_data_of_a_grid(self, condition = None, 
                           x = None, y = None,
                           xrange = None, yrange = None,
                           xlog = None, ylog = None, 
                           validation_frac = None):
        return
    
    
    
    def get_data_of_a_track(self, filename = None, p = None, 
                            fraction = None, random_state = None):
        '''
        To split a track into two subssets for training and testing with a given fraction. 
        The training set has an maxmum number of 1000. 
        outputs:
        training set: gpx, gpy,
        testing set: gpx_v, gpy_v
        '''
        one_track = []
        one_track = pd.read_csv(filename)
        #get rid of the pre-MS
        one_track = one_track.loc[one_track['center_h1'] <= 0.997*np.max(one_track['center_h1']) ]
        
        return
    
    def abc(self):
        one_track = one_track[[x1,x2,y]]
        if (x1log == True): one_track[x1] = np.log10(one_track[x1])
        if (x2log == True): one_track[x2] = np.log10(one_track[x2])        
        if (ylog == True): one_track[y] = np.log10(one_track[y])
        one_track = one_track.replace([np.inf, -np.inf], np.nan)
        one_track.isna().sum()
        one_track = one_track.dropna()
        
        yrange = [np.min(one_track[y]), np.max(one_track[y])]
        x1range = [np.min(one_track[x1]), np.max(one_track[x1])]
        x2range = [np.min(one_track[x2]), np.max(one_track[x2])]
        
        if (ynormalization == True): 
            one_track[y] = (one_track[y] - min(yrange))/(max(yrange) - min(yrange))
        
        if (x1normalization == True): 
            one_track[x1] = (one_track[x1] - min(x1range))/(max(x1range) - min(x1range)) 
        if (xnormalization == True): 
            one_track[x2] = (one_track[x2] - min(x2range))/(max(x2range) - min(x2range)) 
        
        if (random_state == None): random_state = 0
        
        if (len(one_track[y])*fraction >= 1000):
            train = one_track.sample( n = 1000, random_state=random_state) #random state is a seed value
        else:
            train = one_track.sample(frac = fraction, random_state=random_state) #random state is a seed value
        train = train.sort_index()
        test = one_track.drop(train.index)
        test = test.sort_index()
        
        gpy = train[y].to_numpy().reshape(-1, 1)
        gpy = np.float64(gpy)
        gpx1 = train[x1].to_numpy().reshape(-1, 1)        
        gpx1 = np.float64(gpx1)
        gpx2 = train[x1].to_numpy().reshape(-1, 1)        
        gpx2 = np.float64(gpx2)
        
        gpy_v = test[y].to_numpy().reshape(-1, 1)
        gpy_v = np.float64(gpy_v)        
        gpx1_v = test[x1].to_numpy().reshape(-1, 1)
        gpx1_v = np.float64(gpx1_v)
        gpx2_v = test[x2].to_numpy().reshape(-1, 1)
        gpx2_v = np.float64(gpx2_v)
        return gpx1, gpx2, gpy, gpx1_v, gpx2_v, gpy_v, x1range, x2range, yrange
    #############################################################################################################
    
    def preview_2d_data(self, condition = None, number = None,
                        x1 = None, x2 = None ,y = None, 
                        x1log = None, x2log = None, ylog = None, 
                        x1normalization = None, x2normalization = None, ynormalization = None,
                        savefig = None):
        
        if condition == None:
            warnings.warn(f'$condition$ is missing, all csv files in datapath will be used', UserWarning)
            condition = "*.csv"
        if number == None:
            warnings.warn(f'$number$ is missing, 5 file will be used', UserWarning)
            number = 5
            
        if (x == None) or (y == None) or (type(x) != str) or (type(y) != str): 
            raise Warning(f'$X$ and $Y$ must be given with the type of string')
    
        all_files = glob.glob(self._datapath + condition)
        random.shuffle(all_files)
        n = min([int(number), len(all_files)])
        files= all_files[0:n]
        
        plt.figure(figsize=(12, 10))
        plt.xlabel(x)
        plt.ylabel(x2)
        plt.title('Preview of ' + x + ' vs ' + x2 + ' color: ' + y)
        for filename in files:
            print(filename)
            gpx1, gpx2, gpy, gpx1_v, gpx2_v gpy_v, x1range, x2range, yrange = \
            self.get_data_of_a_track(filename, x1, x2, y, x1log, x2log, ylog, \
                                     x1normalization, x2normalization, 
                                     ynormalization, fraction = 0.9)
            plt.scatter(gpx1, gpx2, c = gpy)
        if (savefig == True): plt.savefig(self._savepath + 'S00_' + x1 + '_vs_' + x2 '_vs_' + y + 'preview.png')
        return None
    
    


# global_kernel 

The global_kernel class is to find proper GP kernels for global stellar parameters (e.g. Teff, Radius) and resample them as a function of the age. 

A recomended step-by-step procudure for a stellar parameter is: 
1. 'global_kernel.preview_1d_data'：preview data with to see which kernels could be used;  
2. 'global_kernel.generate_kernel_grid' or manually: set up a list of kernels 
3. 'global_kernel.find_1d_kernel': find the best kernel for a certain parameter with a random selected subset in the grid
4. 'global_kernel.gpmodel': use the kernel found in step 3 and obtain one gp model for each global parameter on each track (as a function of stellar age) 
5. 'global_kernel.resample': resample each evolutaionary track  


# step 1 preview data
The purpose of vasual inspection is find out which kernals may work for the parameter. The effective temperature is the most tricky parameter. Because the curve changes sharply with age at the beginning and the ending parts (pre-MS and giant phases) and is spiky at the turn-off points.

In [ ]:
datadr = '/Users/litz/Documents/GitHub/data/simple_grid_mixed_modes_subset1/'
savedr = '/Users/litz/Documents/GitHub/GPflow/2d-teff/'

In [ ]:
g2 = global_2d_kernel()
g2.set_path(datapath = datadr, savepath = savedr)

# step 1 preview data
The purpose of vasual inspection is find out which kernals may work for the parameter.
The effective temperature is the most tricky parameter. Because the curve changes sharply with age at the beginning and the ending parts (pre-MS and giant phases) and is spiky at the turn-off points. 

In [ ]:
gk.preview_1d_data(condition = '*.csv', number = 8,
                   x = 'star_age', y = 'effective_T', 
                   x2 = False, xlog = False, ylog = False, 
                   xnormalization = True, ynormalization = True,
                   savefig = True)


# Step2 make initial guess of the kernel

individual kernel names are: ['constant','linear','poly2','poly3', 'poly4', 'poly5', 'poly6','cosine','arccosine', 'exponential', 'periodic', 'se', 'rq', 'matern12', 'matern32', 'matern52','static','white']

kernel combinations are also available. Three notes: 1) kernel names and math symbols MUST be seperated by spaces, e.g. 'constant + linear * poly2'; 2) combinations with + - * / are surpported; 3) do not use parentheses or brackets. For the case like '(se + poly3) * linear', please use 'se * linear + ploy3 * linear'

Examples:

kernels =  ['poly3', 'poly4', 'se', 'rq', 'matern12', 'matern32', 'matern52']

kernel_combinations = ['constant + se', 'se + matern12', 'constant + poly3 + se', 'constant + poly3 + rq', 'linear * linear + se'] 

In [ ]:
datadr = '/Users/litz/Documents/GitHub/data/simple_grid_mixed_modes_subset1/'
savedr = '/Users/litz/Documents/GitHub/GPflow/2D-teff/test1/'

gk = global_kernel()
gk.set_path(datapath = datadr, savepath = savedr)

In [ ]:
kernels = ['se', 'rq', 'exponential', 'matern12']
kernel_combinations = [
                       'constant + exponential',
                       'constant + matern12',
                       'se + matern12',
                       'rq + matern12',
                       'se + exponential',
                       'rq + exponential',
                       'matern12 + matern32',
                       'matern12 + exponential',
                       'matern12 + matern12',
                       'exponential + exponential'
                      ]

We could also check kernel functions with following codes


In [ ]:
def plotkernelfunction(k = None, ax = None, xmin=None, xmax=None, other=None, kname = None):
    xx = np.linspace(xmin, xmax, 200)[:,None]
    ax.plot(xx, k(xx, np.zeros((1,1)) + other))
    ax.set_title(kname)

for kname in kernels:
    kernel = gk.kernel_bank(kname)
    f, axes = plt.subplots(1, 1, figsize=(6, 5), sharex=True)
    plotkernelfunction(k = kernel, ax = axes, xmin=-3, xmax=3, other=1.0, kname = kname)
    
for kname in kernel_combinations:
    kernel = gk.solve_kernel_combinations(kname)
    f, axes = plt.subplots(1, 1, figsize=(6, 5), sharex=True)
    plotkernelfunction(k = kernel, ax = axes, xmin=-3, xmax=3, other=1.0, kname = kname)

# Step 3 use a subset to test kernels

In [ ]:
start_time = time.time()

teff_offset = gk.find_1d_kernel(condition = '*.csv',
                                subset_fraction = 0.2, 
                                x = 'star_age', y = 'effective_T', 
                                x2 = False, xlog = False, ylog = False,
                                xnormalization = True,
                                ynormalization = True,
                                kernels = kernels, 
                                kernel_combinations = kernel_combinations,
                                iterations = 10, validation_frac = 0.4,
                                printinfo = False, figures = True)

print("--- %s seconds ---" % (time.time() - start_time))

In [ ]:
teff_offset

In [ ]:
teff_offset.to_csv(savedr + 'teff_kernel_test.csv')


We got several kernels which look promising. They are

exponential, matern12, constant + exponetial, constant + matern12, matern12 + matern12, exponential + exponential, matern12 + exponential

We may want a second match with only those kernels but more files and see who wins. 

In [ ]:
datadr = '/Users/litz/Documents/GitHub/data/simple_grid_mixed_modes_subset1/'
savedr = '/Users/litz/Documents/GitHub/GPflow/teff/test2/'

gk = global_kernel()
gk.set_path(datapath = datadr, savepath = savedr)


kernels = ['exponential' ,'matern12']
kernel_combinations = [
                       'constant + exponential',
                       'constant + matern12',
                       'matern12 + exponential',
                       'matern12 + matern12',
                       'exponential + exponential'
                      ]



start_time = time.time()

teff_offset_2 = gk.find_1d_kernel(condition = '*.csv',
                                subset_fraction = 0.3, 
                                x = 'star_age', y = 'effective_T', 
                                x2 = False, xlog = False, ylog = False,
                                xnormalization = True,
                                ynormalization = True,
                                kernels = kernels, 
                                kernel_combinations = kernel_combinations,
                                iterations = 10, validation_frac = 0.5,
                                printinfo = False, figures = True)

print("--- %s seconds ---" % (time.time() - start_time))

In [ ]:
teff_offset_2.to_csv(savedr + 'teff_kernel_test_2.csv')

In [ ]:
teff_offset_2

In [ ]:
np.mean(teff_offset_2)

In [ ]:
np.std(teff_offset_2)

# Step 4 Apply the best GP kernel to the whole grid

We found that 'matern12' is the best and the most stable kernel. We hence apply it to the whole grid. 

gk.apply_1d_kernel uses an input kernel to model a stellar parameter as a function of the other parameter for each evolutionary track. The output is one GP model per track. 

gk.apply_1d_kernel returns a pandas.dataframe which content the deviation between the GP model and the data for each evolutionary track. The dataframe looks like: 

-------------
index,     evolutionary_track,       x,          y,        xlog,    ylog,     xrange,       yrange,         xnormalization,     ynormalization,     mean_deviation,     max_deviation,        saveGPmodel         

  1,      m1.0_feh2.0_MLT...,   star_age,   effective_T,    True,    True,  [-4.5, 10.],   [3.6,3.8] ,       Ture ,     Ture,      1.0d-8,            1.0d-4,     GPmodel-effective_T...
  ......

validate_mean and validate_sigma is the mean and the standard devitation of the probability distribution of residuals between model and data. The two parameters will be used to estimate the uncertainty of a predication value. 

xrange and yrange gives a parameter space in which the derived GP model can be trust. Obviously, this space can not exceed the min/max of x and y values. To avoid edge effects, we removed edge spaces where model predictions is 3sigma away from validations.      
  

In [ ]:
datadr = '/Users/litz/Documents/GitHub/data/simple_grid_mixed_modes_subset1/'
savedr = '/Users/litz/Documents/GitHub/GPflow/teff/'
gk = global_kernel()
gk.set_path(datapath = datadr, savepath = savedr)


kernels = ['exponential + exponential']

In [ ]:
start_time = time.time()

gk_outputs = gk.apply_1d_kernel(condition = '*.csv',
                                      x = 'star_age', y = 'effective_T', x2 = False, 
                                      xlog = False, ylog = False,
                                      xnormalization = True, ynormalization = True,
                                      validation_frac = 0.4,
                                      kernels = kernels, 
                                      combination = True,
                                      iterations = 10,
                                      savemodelprefix = 'GPmodel-effective_T-',
                                      printinfo = False, 
                                      figures = True,
                                      savetablename = 'gk_outputs.csv')

print("--- %s seconds ---" % (time.time() - start_time))


In [ ]:
gk_outputs.head()

In [ ]:
#offsets = 10**(3.7 + gk_outputs['mean_deviation']) - 10**3.700000

offsets = gk_outputs['mean_deviation']
print(np.mean(offsets), 
      np.min(offsets), 
      np.max(offsets))

In [ ]:
#offsets = 10**(3.7 + gk_outputs['max_deviation']) - 10**3.700000

offsets = gk_outputs['max_deviation']

print(np.mean(offsets), 
      np.min(offsets), 
      np.max(offsets))

# Step 5 Use models to predict/interpolate 

In [ ]:
datadr = '/Users/litz/Documents/GitHub/data/simple_grid_mixed_modes_subset1/'
savedr = '/Users/litz/Documents/GitHub/GPflow/teff/'
gk = global_kernel()
gk.set_path(datapath = datadr, savepath = savedr)

In [ ]:
gptablename = savedr + 'GPmodel-effective_T-gk_outputs.csv'
gpdictpath = savedr + 'GPmodel-effective_T-dicts/'
gpdatapath = savedr + 'GPmodel-effective_T-dicts/'

In [ ]:
dtype = {'saveGPmodel': object} #, 'saveGPmodel': str, 'saveGPdata': str}

gptable = gk.load_gp_bank(tablename = gptablename, dtype=dtype, gpdictpath = gpdictpath, gpdatapath = gpdatapath)

In [ ]:
trackname = 'm1.53_feh3.0_MLT1.9_fov0.018'
tablerow = gptable.loc[gptable['evolutionary_track'] == trackname] 
tablerow.head()


In [ ]:
eval(tablerow['xrange'].all())

In [ ]:
#x_input need to have same unit with the data grid!
# parameter    unit
#   age       year
#  Teff        K
#  logg       dex
#  radius     solar
#luminosity   solar
#frequency    microHz

x_input = np.linspace(2e9, 3e9, num=100)

track_x, track_y, \
track_x_v, \
track_y_v, track_y_v_e, \
x_new, \
y_new, y_new_e = gk.use_gp_model(model_dict = tablerow['saveGPmodel'].all(), 
                                  model_data = tablerow['saveGPdata'].all(), 
                                  x = tablerow['x'].all(), y = tablerow['y'].all(), 
                                      x2 = tablerow['x2'].all(), 
                                      xlog = tablerow['xlog'].all(), ylog = tablerow['ylog'].all(),
                                      xrange = eval(tablerow['xrange'].all()), 
                                      yrange = eval(tablerow['yrange'].all()),
                                      xnormalization = tablerow['xnormalization'].all(), 
                                      ynormalization = tablerow['ynormalization'].all(),
                                      kernels = [tablerow['kname'].all()], 
                                      combination = tablerow['combination'].all(),
                                      x_input = x_input)




In [ ]:
plt.plot(track_x/1.0e9, track_y, 'k')
plt.errorbar(x_new/1.0e9, y_new, yerr = y_new_e, c = 'r', marker = '.')
plt.xlabel('Age (Gyr)')
plt.ylabel(r'Teff (K)')

In [ ]:
plt.plot(track_x_v/1.e9, track_y_v_e, 'b-')
plt.plot(x_new/1.e9, y_new_e, 'k.')
plt.xlabel('Age (Gyr)')
plt.ylabel(r'err_Teff (K)')


In [ ]:
10000/(32/20)/60/24